/* THIS CODE IS MY OWN WORK, IT WAS WRITTEN WITHOUT CONSULTING CODE WRITTEN BY OTHER STUDENTS. Asuka Li */

Collaboration Statement
I have consulted the following through writing this program:
- lecture slidies for CS 470
- asaini's Apriori project on Github

In [10]:
def apriori(inp, minSup, out):
    import pandas as pd
    import numpy as np
    from itertools import combinations
    
    # helper functions
    def k_itemsets(tmp, k):
        return list(combinations(tmp,k))
    
    
    data = pd.read_csv(inp)
    keywords = [x.split(';') for x in data['text_keywords']] # Set of instances in relevant column
    unqWords = set([item for sublist in keywords for item in sublist])

    hmap = {} # stores the output result
    tmphmap = {} # stores the frequency of each round, needs to be reset
    k = 1
    # count support of 1 itemsets 
    for word in unqWords:
        for list_kw in keywords:
            if word in list_kw:
                tmphmap[word] = tmphmap.get(word, 0) + 1
                
    # eliminate candidates that are infrequent for 1 itemsets
    infreq = []
    newlist = [] # stores the frequent itemsets in each round
    for word in tmphmap:
        if tmphmap[word] <= minSup:
            infreq.append(word)
        else:
            hmap[word] = tmphmap.get(word)
            newlist.append(word)
                
               
    while (len(newlist) > 0):
        # generate length (k+1) candidate itemsets
        if (k == 1):
            # uses unqWords since building from the individual words
            itemsets = k_itemsets(newlist, k+1)
        else:
            newlist = set([item for sublist in newlist for item in sublist])
            itemsets = k_itemsets(newlist, k+1)
        
        # prune candidate itemsets containing subsets of length k that are infrequent
        for x in infreq:
            candidates = [item for item in itemsets if not (x in item or set(x).issubset(item))]

        # Count the support of each new candidate 
        tmphmap = {}
        for item in candidates:
            for kw in keywords:
                if set(item).issubset(kw):
                    tmphmap[item] = tmphmap.get(item, 0)+1
        
        # eliminate candidates that are infrequent
        infreq = []
        newlist = []
        for word in tmphmap:
            if tmphmap[word] <= minSup:
                infreq.append(word)
            else:
                hmap[word] = tmphmap.get(word)
                newlist.append(word)
         
        k += 1
    
    import operator
    hmap = dict(sorted(hmap.items(), key=operator.itemgetter(1),reverse=True))
    
    # writing to file
    with open(out, 'w') as f:
        for key, value in hmap.items():
            if type(key) == str:
                f.write('%s ' % key)
            else:
                for k in key:
                    f.write('%s ' % k)
            f.write('(%s)\n' % value)
    

In [11]:
apriori("data.csv", 500, "output.txt")